In [ ]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import numpy as np
import cv2

## No Transfer Learning

### Processing Data into Batches

In [ ]:
data = tf.keras.preprocessing.image_dataset_from_directory('data', batch_size=32)

In [ ]:
data = data.map(lambda x,y: (x/255,y))
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()
batch

### Plot Pictures

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx,img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

#### Separating Data

In [ ]:
train_size = int(len(data)*0.6)
test_size = int(len(data)*0.2)
val_size = int(len(data)*0.2)

In [ ]:
train = data.take(train_size)
test = data.take(test_size)
val = data.take(val_size)

### Create Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16,(3,3),activation='relu',input_shape=(256,256,3))) 
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32,(3,3),activation='relu')) 
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

### Train Model

In [ ]:
tensor_cb = tf.keras.callbacks.TensorBoard(log_dir='logs')
hist = model.fit(train, epochs=50, validation_data=val, callbacks=[tensor_cb])

In [ ]:
import cv2
import numpy as np
test_img1 = cv2.imread('jaguar.jpg')
result = int(model.predict(np.expand_dims(test_img1/255,0)) > 0.5)
result

In [ ]:
test_img1 = cv2.imread('chee.jpg')
result = int(model.predict(np.expand_dims(test_img1/255,0)) > 0.5)
result

In [ ]:
test_img1 = cv2.imread('chee.jpg')
result = int(model.predict(np.expand_dims(test_img1/255,0)) > 0.5)
result

In [ ]:
model.evaluate(test)

## Transfer Learning

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os

In [ ]:
base_model = MobileNetV2(
    input_shape=(256, 256, 3),
    include_top=False,  
    weights='imagenet'
)

base_model.trainable = False  


In [ ]:
model = Sequential()

In [ ]:
model.add(base_model)
model.add(GlobalAveragePooling2D())

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
tensor_cb = tf.keras.callbacks.TensorBoard(log_dir='logs')
hist = model.fit(train, epochs=35, validation_data=val, callbacks=[tensor_cb])

In [ ]:
model.evaluate(test)

In [ ]:
model.summary()

In [ ]:
model.save('cheetah_jaguar_transfer_model.keras')